In [1]:
import numpy as np
from pyscf import gto, lib
from pyscf.gto.eval_gto import make_screen_index, _GTO_EVAL_FUNCTIONS

In [2]:
_GTO_EVAL_FUNCTIONS["GTOval_ipr"] = (9, 9)
_GTO_EVAL_FUNCTIONS["GTOval_iprc"] = (9, 9)

## Definition

In [3]:
basis = {
    "C": "cc-pVTZ",
    "H": "cc-pVTZ",
    "Sb": "cc-pVTZ-PP",
}

basis_ecp = {
    "Sb": "cc-pVTZ-PP",
}

mol = gto.Mole(atom="""
Sb        -1.33937843      0.44597852     -1.27279684
Sb         1.33937843     -0.44597852     -1.27279684
C         -1.40429524      1.10441871      0.83468205
C         -2.16210130     -1.56132398     -0.84717555
C          2.16210130      1.56132398     -0.84717555
C          1.40429524     -1.10441871      0.83468205
H         -0.69918639      1.91987631      1.00872018
H         -1.16111477      0.29030616      1.51873028
H         -2.40124532      1.47235562      1.08516843
H         -2.02002046     -2.22909286     -1.69887295
H         -1.69052287     -2.01612927      0.02577778
H         -3.23450854     -1.49489801     -0.65423339
H          2.02002046      2.22909286     -1.69887295
H          3.23450854      1.49489801     -0.65423339
H          1.69052287      2.01612927      0.02577778
H          0.69918639     -1.91987631      1.00872018
H          2.40124532     -1.47235562      1.08516843
H          1.16111477     -0.29030616      1.51873028
""", basis=basis, ecp=basis_ecp).build()

In [4]:
x = np.arange(2048)
coords_2 = np.stack([np.sin(x), np.cos(x), np.sin(x + 0.5)], axis=1)

In [5]:
x = np.arange(131072)
coords_128 = np.stack([np.sin(x), np.cos(x), np.sin(x + 0.5)], axis=1)

In [6]:
x = np.arange(1048576)
coords_1k = np.stack([np.sin(x), np.cos(x), np.sin(x + 0.5)], axis=1)

In [7]:
shls_slice = [sh0, sh1] = [4, 126]

## Usual case

In [8]:
ao = mol.eval_gto("GTOval_sph", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(-291.50611912900354, [2048, 366])

In [9]:
ao = mol.eval_gto("GTOval_cart", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(-109.46020421466841, [2048, 410])

In [10]:
ao = mol.eval_gto("GTOval_ig_sph", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(180.75613409086753, [2048, 366, 3])

## With args

In [11]:
ao = mol.eval_gto("GTOval_sph", coords_2, cutoff=1e-4).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "sph")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(20.063290707360913, [2048, 349])

In [12]:
ao = mol.eval_gto("GTOval_sph_deriv1", coords_2, cutoff=1e-2).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "sph")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(228.2644447105396, [2048, 349, 4])

In [13]:
ao = mol.eval_gto("GTOval_cart_deriv1", coords_2, cutoff=1e-2).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "cart")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(-427.8204853612391, [2048, 392, 4])